## 4.6 GPU计算

在本节中，我们将介绍如何使用单块 NVIDIA GPU来计算。

In [2]:
!nvidia-smi

Thu Jun 03 19:15:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 385.54                 Driver Version: 385.54                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1050   WDDM  | 00000000:01:00.0 N/A |                  N/A |
| 30%   30C    P8    N/A /  N/A |    574MiB /  2048MiB |     N/A      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

### 4.6.1 计算设备

MXNet可以指定用来存储和计算的设备，如使用内存的CPU或者使用显存的。默认情况下，MXNet会将数据创建在内存，然后使用CPU来计算。在MXNet中，mx.cpu()（或者在括号里填任意整数）表示所有的物理CPU和内存。这意味着，MXNet的计算会尽量使用所有的CPU核。但mx.gpu()只代表一块GPU和相应的显存。如果有多块GPU，我们用mx.gpu(i)来表示第$i$块GPU及相应的显存($i$从0开始)且mx.gpu(0)和mx.gpu()等价。

In [3]:
import mxnet as mx
from mxnet import nd
from mxnet.gluon import nn

mx.cpu(), mx.gpu(), mx.gpu(1)

(cpu(0), gpu(0), gpu(1))

### 4.6.2 NDArray的GPU计算

In [4]:
x = nd.array([1, 2, 3])
x


[1. 2. 3.]
<NDArray 3 @cpu(0)>

In [5]:
x.context

cpu(0)

#### GPU上的存储

In [9]:
a  = nd.array([1, 2, 3], ctx=mx.gpu())
a


[1. 2. 3.]
<NDArray 3 @gpu(0)>

下⾯我们将内存上的NDArray变量x复制到gpu(0)上

In [10]:
y = x.copyto(mx.gpu())
y


[1. 2. 3.]
<NDArray 3 @gpu(0)>

In [11]:
z = x.as_in_context(mx.gpu())
z


[1. 2. 3.]
<NDArray 3 @gpu(0)>

In [12]:
y.as_in_context(mx.gpu()) is y

True

In [13]:
y.copyto(mx.gpu()) is y

False

#### GPU上的计算

In [14]:
(z + 2).exp() + y


[ 21.085537  56.59815  151.41316 ]
<NDArray 3 @gpu(0)>

### 4.6.3 Gluon的GPU计算

In [15]:
net = nn.Sequential()
net.add(nn.Dense(1))
net.initialize(ctx=mx.gpu())

In [17]:
net(y)


[[0.0068339 ]
 [0.01366779]
 [0.02050169]]
<NDArray 3x1 @gpu(0)>

In [18]:
net[0].weight.data()


[[0.0068339]]
<NDArray 1x1 @gpu(0)>

### 小结

- MXNet可以指定⽤来存储和计算的设备，如使⽤内存的CPU或者使⽤显存的GPU。在默认情况下， MXNet会将数据创建在内存，然后利⽤CPU来计算。
- MXNet要求计算的所有输⼊数据都在内存或同⼀块显卡的显存上。

### 练习

- 试试⼤⼀点⼉的计算任务，如⼤矩阵的乘法，看看使⽤CPU和GPU的速度区别。如果是计算量很小的任务呢？

⼤⼀点⼉的计算任务，应该GPU比CPU快。计算量很小时，应该差不多。

- GPU上应如何读写模型参数？

In [19]:
# 写
filename = 'net.params'
net.save_parameters(filename)

In [22]:
net2 = nn.Sequential()
net2.add(nn.Dense(1))
# 读
net2.load_parameters(filename, ctx=mx.gpu())

In [23]:
net2(y)


[[0.0068339 ]
 [0.01366779]
 [0.02050169]]
<NDArray 3x1 @gpu(0)>

In [24]:
net2[0].weight.data()


[[0.0068339]]
<NDArray 1x1 @gpu(0)>